In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew
from openai import AzureOpenAI

In [3]:
import os
from utils_azure import get_azure_credentials

azure_key, azure_endpoint = get_azure_credentials()
os.environ['AZURE_KEY'] = azure_key
os.environ["AZURE_ENDPOINT"] = azure_endpoint


In [4]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint=os.getenv("AZURE_ENDPOINT")
)

In [5]:
researcher = Agent(
    role="Digital Marketing Researcher",
    goal="Gather accurate, relevant, and engaging information on {topic} to support the creation of a high-quality blog post.",
    backstory="You are an expert in digital marketing research, specializing in identifying trends, data, and insights that resonate with the target audience. "
              "Your role is to collect information that is factual, up-to-date, and valuable to the audience. "
              "Your research will serve as the foundation for the Content Writer to create an engaging and informative blog post.",
    instructions=[
        "1. Identify the target audience and their preferences for the topic: {topic}.",
        "2. Conduct thorough research using credible sources such as industry reports, case studies, and reputable websites.",
        "3. Gather data, statistics, and trends that support the topic and provide value to the audience.",
        "4. Organize the research into clear sections (e.g., introduction, key points, examples, and conclusions).",
        "5. Ensure all information is accurate, up-to-date, and properly cited.",
        "6. Provide a summary of the research findings to the Content Writer for further development."],
    llm_config = {
        "client": client,
        "model": "DeepSeek-R1-ozfcc"
    },
    allow_delegation=False,
	verbose=True
)

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write a compelling, engaging, and well-structured blog post on {topic} based on the research provided.",
    backstory="You are a skilled content writer with expertise in crafting blog posts that resonate with the target audience. "
              "Your role is to transform the research findings into a clear, engaging, and informative article that aligns with the company's tone and style.",
    instructions=[
        "1. Review the research provided by the Researcher and identify the key points to include in the blog post.",
        "2. Write a captivating headline and introduction to grab the reader's attention.",
        "3. Structure the blog post into sections (e.g., introduction, main body, conclusion) for easy readability.",
        "4. Use clear, concise, and engaging language to communicate the information effectively.",
        "5. Incorporate relevant examples, statistics, and quotes from the research to support your points.",
        "6. Ensure the tone and style align with the company's brand guidelines.",
        "7. Optimize the content for SEO by including relevant keywords and meta descriptions.",
        "8. Submit the draft to the Editor for review and refinement."
    ],
    llm_config = {
        "client": client,
        "model": "DeepSeek-R1-ozfcc"
    },
    allow_delegation=False,
    verbose=True
)

In [7]:
editor = Agent(
    role="Content Editor",
    goal="Review and refine the blog post to ensure it is polished, error-free, appropiate for the company's policy and ready for publication.",
    backstory="You are an experienced content editor with a keen eye for detail and a deep understanding of the company's brand voice. "
              "Your role is to ensure the blog post is clear campared to the {topic}, engaging, and free of errors before it is published.",
    instructions=[
        "1. Review the blog post draft for grammar, spelling, and punctuation errors.",
        "2. Ensure the content is clear, concise, and easy to understand.",
        "3. Verify that the tone and style align with the company's brand guidelines.",
        "4. Check that the blog post is well-structured and flows logically.",
        "5. Confirm that all facts, statistics, and quotes are accurate and properly cited.",
        "6. Provide constructive feedback to the Content Writer if revisions are needed.",
        "7. Finalize the blog post and prepare it for publication."
    ],
    llm_config = {
        "client": client,
        "model": "DeepSeek-R1-ozfcc"
    },
    allow_delegation=False,
    verbose=True
)

# Tools

In [8]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool,  WebsiteSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Tasks

In [9]:
plan = Task(
    description=(
        "1. Analyze the topic: {topic} and identify its relevance to the target audience.\n"
        "2. Research the latest trends, statistics, and news related to {topic}.\n"
        "3. Identify key players, influencers, or brands in the industry related to {topic}.\n"
        "4. Gather credible sources such as industry reports, case studies, and reputable websites.\n"
        "5. Highlight pain points, challenges, and opportunities related to {topic}.\n"
        "6. Organize the research into clear sections (e.g., trends, data, examples, and sources)."
    ),
    expected_output=(
        "A detailed research document containing:\n"
        "- Key trends and statistics related to {topic}.\n"
        "- Credible sources and references.\n"
        "- Insights into the target audience's pain points and interests.\n"
        "- A structured outline of the research findings."
    ),
    tools = [search_tool, scrape_tool],
    agent=researcher,
)

In [10]:
audience_analysis = Task(
    description=(
        "1. Define the target audience for the blog post on {topic}.\n"
        "2. Identify the audience's demographics, interests, and preferences.\n"
        "3. Analyze the audience's pain points and how {topic} can address them.\n"
        "4. Determine the tone and style that will resonate with the audience.\n"
        "5. Provide recommendations for engaging the audience effectively."
    ),
    expected_output=(
        "An audience analysis report containing:\n"
        "- Demographics and psychographics of the target audience.\n"
        "- Pain points and interests related to {topic}.\n"
        "- Recommendations for tone, style, and engagement strategies."
    ),
    tools = [search_tool, scrape_tool],
    agent=researcher,
)

In [11]:
# writer task

write = Task(
    description=(
        "1. Use the research document and audience analysis to create a blog draft.\n"
        "2. Write a compelling headline and introduction to grab the reader's attention.\n"
        "3. Structure the blog post into sections (e.g., introduction, main body, conclusion).\n"
        "4. Incorporate key points, examples, and statistics from the research.\n"
        "5. Use a tone and style that aligns with the target audience and brand guidelines.\n"
        "6. Include a clear call-to-action (CTA) to engage the audience.\n"
        "7. Optimize the content for SEO by including relevant keywords and meta descriptions."
    ),
    expected_output=(
        "A complete blog draft containing:\n"
        "- A captivating headline and introduction.\n"
        "- Well-structured sections with clear key points.\n"
        "- Examples, statistics, and quotes from the research.\n"
        "- A strong call-to-action (CTA).\n"
        "- SEO-optimized content with keywords and meta descriptions."
    ),
    tools = [search_tool, scrape_tool],
    agent=writer,
)

In [12]:
revise_blog = Task(
    description=(
        "1. Review the Editor's feedback on the blog draft.\n"
        "2. Make necessary revisions to improve clarity, tone, and structure.\n"
        "3. Ensure all facts, statistics, and quotes are accurate and properly cited.\n"
        "4. Double-check SEO optimization, including keywords and meta descriptions.\n"
        "5. Submit the revised draft to the Editor for final approval."
    ),
    expected_output=(
        "A revised blog draft that incorporates all feedback and is ready for final review."
    ),
    tools = [search_tool, scrape_tool],
    agent=writer,
)

In [13]:
# editot task

edit = Task(
    description=(
        "1. Review the blog draft for grammar, spelling, and punctuation errors.\n"
        "2. Ensure the content is clear, concise, and easy to understand.\n"
        "3. Verify that the tone and style align with the company's brand guidelines.\n"
        "4. Check that the blog post is well-structured and flows logically.\n"
        "5. Confirm that all facts, statistics, and quotes are accurate and properly cited.\n"
        "6. Provide constructive feedback to the Writer for revisions if needed."
    ),
    expected_output=(
        "A reviewed blog draft with:\n"
        "- Corrections for grammar, spelling, and punctuation.\n"
        "- Suggestions for improving clarity, tone, and structure.\n"
        "- Confirmation of factual accuracy and proper citations.\n"
        "- Feedback for the Writer if revisions are required."
    ),
    tools = [search_tool, scrape_tool],
    agent=editor,
)

In [14]:
finalize_blog = Task(
    description=(
        "1. Review the revised blog draft from the Writer.\n"
        "2. Ensure all feedback has been incorporated and the content is error-free.\n"
        "3. Confirm that the blog post is optimized for SEO and aligns with brand guidelines.\n"
        "4. Prepare the blog post for publication by formatting it appropriately.\n"
        "5. Submit the finalized blog post for publishing."
    ),
    expected_output=(
        "A finalized blog post that is:\n"
        "- Error-free and polished.\n"
        "- SEO-optimized and aligned with brand guidelines.\n"
        "- Ready for publication."
    ),
    tools = [search_tool, scrape_tool],
    agent=editor,
)

# Crew

In [15]:
crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[plan, audience_analysis, write, revise_blog, edit, finalize_blog],  # tasks oprates swquentially
    verbose=True,  # should be boolean
    memory = True
)

# Run Crew

In [17]:
from IPython.display import Markdown
topic = "Write a blog post about Super ball 2025"
result = crew.kickoff(inputs={"topic": topic})
Markdown(result.raw)

# Agent: Digital Marketing Researcher
## Task: 1. Analyze the topic: Write a blog post about Super ball 2025 and identify its relevance to the target audience.
2. Research the latest trends, statistics, and news related to Write a blog post about Super ball 2025.
3. Identify key players, influencers, or brands in the industry related to Write a blog post about Super ball 2025.
4. Gather credible sources such as industry reports, case studies, and reputable websites.
5. Highlight pain points, challenges, and opportunities related to Write a blog post about Super ball 2025.
6. Organize the research into clear sections (e.g., trends, data, examples, and sources).


# Agent: Digital Marketing Researcher
## Thought: I need to gather updated and relevant information about the Super Bowl 2025, which includes trends, statistics, key players, brands, and insights into the target audience's pain points and interests. This will help in creating a comprehensive blog post.
## Using tool: Search the

# Super Bowl 2025: Trends, Insights, and Key Highlights 

## The Countdown to an Unforgettable Match 
The Super Bowl is more than just a football game; it’s a cultural phenomenon. Super Bowl 2025 promises to deliver an exhilarating matchup between the Kansas City Chiefs and the Philadelphia Eagles, captivating the attention of millions around the globe. Anticipation builds as this iconic event is set to unfold on **February 9, 2025**, and fans are gearing up for what is expected to be one of the most-watched sporting occasions in history.

## Eye-Catching Trends and Statistics
### Phenomenal Viewership
An estimated **127.7 million viewers** tuned into the previous Super Bowl, which set a significant record in television history. Projections for Super Bowl 2025 suggest viewership could reach up to **150 million** globally. This demonstrates a growing trend in the NFL's popularity and engagement across diverse demographics.

### Ticket Costs and Economic Impact
With tremendous excitement comes significant expenses. As of now, ticket prices for Super Bowl 2025 start around **$5,122**, reflecting a **2% increase** from the last meeting between these two teams in 2023. For those seeking a premium experience, suites are priced significantly higher, with many exceeding **$500,000**. This surge in ticket prices is expected to contribute to total spending projected at around **$18.6 billion** on food, beverages, and merchandise across the event, averaging **$91.58** per person.

### Accommodations on the Rise
Demand for accommodations has soared, with **Airbnb** experiencing a **400% increase** in searches related to New Orleans since the matchup announcement. This underscores the economic impact the Super Bowl has on local economies, as fans from various regions are seeking both traditional hotel venues and alternative accommodation options via platforms like Airbnb and VRBO.

## Audience Insights: The Super Bowl Demographic
### Betting Trends
Super Bowl 2025 is anticipated to see about **68 million American adults** placing bets, marking a record high, according to the American Gaming Association. This presents brands with a significant opportunity to engage with both bettors and casual fans, utilizing promotional strategies that resonate with this audience.

### Spending Behavior
Fans are projected to heavily indulge in traditional game-day favorites, including chicken wings, nachos, and beverages. Spending on these essentials reflects just how pivotal the Super Bowl is in driving consumer behavior and hospitality sales, benefitting sponsors and vendors alike.

## Challenges and Opportunities
### Expensive Tickets: A Double-Edged Sword  
While high ticket prices may deter some fans from attending the game in-person, this could lead to increased interest in online streaming and at-home viewing parties, reshaping how the event is consumed. Brands should pivot toward digital and experiential engagement strategies to capture this segment of fans.

### Technology Meets Tradition
The rise of social media platforms and mobile applications is transforming the interaction fans have with the game. Brands that effectively harness technology for engagement can better reach their target demographics, creating opportunities through interactive promotions and advertisements.

## Celebrities and Influencers Take the Spotlight
Expectations run high for celebrity appearances, which will amplify the spectacle of Super Bowl 2025. High-profile figures, from sports luminaries to major celebrities, are slated to enhance social media buzz, generating viewer interest alongside promotional campaigns from significant brands like Nike and Budweiser.

## Conclusion: Embracing the Excitement of Super Bowl 2025
Super Bowl 2025 is more than just a football game; it’s an exhilarating event that captures the hearts of millions. As viewership and participation continue to soar, brands must leverage this engagement creatively, providing promotions and content that resonate with the spirit of the occasion.

## Call to Action
Are you ready to join in on the excitement? Whether you plan to host a party or bet on your favorite team, share your Super Bowl plans with us! Don’t forget to follow us for the latest insights and updates leading up to the big game.

**SEO Keywords**: Super Bowl 2025, NFL trends, Chiefs vs Eagles, Super Bowl viewership, game day spending, celebrity appearances.

**Meta Description**: Explore the exciting trends and key statistics surrounding Super Bowl 2025, featuring a spectacular matchup between the Kansas City Chiefs and Philadelphia Eagles, along with insights on viewership, ticket costs, and engaging fan experiences.

**References**:
- "Super Bowl Trends 2025" - Forbes - [Link](https://www.forbes.com/sites/jefffromm/2025/02/04/super-bowl-2025-trends-how-ticket-costs-and-viewership-are-shaping-up/)
- "Super Bowl LIX Makes TV History with Over 127 Million Viewers" - Nielsen - [Link](https://www.nielsen.com/news-center/2025/super-bowl-lix-makes-tv-history-with-over-127-million-viewers/)

This finalized blog post incorporates up-to-date information, ensuring clarity and cohesiveness. It has been optimized for SEO while aligning with brand guidelines, making it ready for publication.
```

git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin __link__
git push -u origin main